Install notebook dependencies.

In [ ]:
import sys
!{sys.executable} -m pip install ipynbname category_encoders matplotlib boto3 mlflow==1.28.0

Import dependencies into our session.

In [ ]:
from pprint import pprint
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import mlflow
import os
import matplotlib.pyplot as plt
import urllib3

Connect to the MLFlow server instance. We setup an EXPERIMENT_NAME and turn on the mlflow client auto logging feature.

In [ ]:
HOST = "http://mlflow:5500"
EXPERIMENT_NAME = "TestModel"

# Connect to MLflow tracking server
mlflow.set_tracking_uri(HOST)
mlflow.set_experiment(EXPERIMENT_NAME)
# enable autologging
mlflow.sklearn.autolog(log_input_examples=True)

Define a helper method we will use to return the model data once it has run.

In [ ]:
def fetch_logged_data(run_id):
    m_client = mlflow.tracking.MlflowClient()
    data = m_client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in m_client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags, artifacts

Create some random data for our linear regression plot.

In [ ]:
m = 100
np.random.seed(365)
x = 6 * np.random.rand(m, 1) - 3
x = np.sort(x, axis=0)  # sort the values
y = 0.5 * x**2 + x + 2 + np.random.randn(m, 1)

Use the sklearn LinearRegression model to train and fit our random data sample. We also store the plotted figure of the results in MLFlow.

In [ ]:
model = LinearRegression()
with mlflow.start_run() as run:
    model.fit(x, y)
    fig, ax = plt.subplots()
    ax.plot(x, y, "b.")
    ax.plot(x, model.predict(x), "r-")
    mlflow.log_figure(fig, "plot.png")

Print out the shape, metrics and parameters of our simple linear regression model.

In [ ]:
print(x.shape)

In [ ]:
print(y.shape)

In [ ]:
params, metrics, tags, artifacts = fetch_logged_data(run.info.run_id)

In [ ]:
pprint(params)

In [ ]:
pprint(metrics)

In [ ]:
pprint(tags)

In [ ]:
pprint(artifacts)

Predict the Y output given a simple one dimension X input.

In [ ]:
model.predict([[1.23]])